In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import glob

In [3]:
tf.__version__

'2.0.0'

In [4]:
image_filenames = glob.glob('./dc/train/*.jpg') #提取路径

In [5]:
image_filenames = np.random.permutation(image_filenames) #对路径进行乱序

In [6]:
image_filenames

array(['./dc/train\\cat.7264.jpg', './dc/train\\dog.3809.jpg',
       './dc/train\\cat.2781.jpg', ..., './dc/train\\cat.6730.jpg',
       './dc/train\\cat.6858.jpg', './dc/train\\cat.12073.jpg'],
      dtype='<U24')

In [5]:
lables = list(map(lambda x: float(x.split('\\')[1].split('.')[0] == 'cat'), image_filenames))
#提取对应的lable

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((image_filenames, lables))

In [7]:
dataset

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.float32)>

In [8]:
#定义一个函数处理图片
def _pre_read(img_filename, lable):
    image = tf.io.read_file(img_filename) #加载图片
    image = tf.image.decode_jpeg(image, channels=3) #解码
    image = tf.image.resize(image, (200, 200))#统一定义大小200*200
    image = tf.reshape(image, [200,200,3]) 
    image = tf.image.per_image_standardization(image) #图片数据标准化
    return image, lable

In [9]:
dataset = dataset.map(_pre_read) #转换

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [10]:
dataset = dataset.shuffle(300)

In [11]:
dataset = dataset.repeat()

In [12]:
dataset = dataset.batch(32)

In [13]:
dataset

<BatchDataset shapes: ((None, 200, 200, 3), (None,)), types: (tf.float32, tf.float32)>

In [14]:
model = keras.Sequential()

In [15]:
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(200, 200, 3)))

In [16]:
model.add(layers.MaxPooling2D((2, 2))) # 默认参数是2*2，会将图片缩小为原来的一半

In [17]:
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [18]:
model.add(layers.MaxPooling2D((2, 2)))

In [19]:
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [20]:
model.add(layers.MaxPooling2D((2, 2)))

In [21]:
model.add(layers.Flatten())

In [22]:
model.add(layers.Dense(64, activation='relu'))

In [23]:
model.add(layers.Dense(1, activation='sigmoid'))

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 33856)             0

In [25]:
#第四节 6.4

In [26]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'] #metrics可以监控很多指标，这里监控的是acc
)

In [ ]:
history = model.fit(dataset, epochs=10, steps_per_epoch=781) #因为前面对dataset进行了repeat，所以这里需要设置steps_per_epoch参数（len(image_filenames)/batch）

Train for 781 steps
Epoch 1/10
781/781 [==============================] - 1263s 2s/step - loss: 0.5798 - acc: 0.6847
Epoch 2/10
781/781 [==============================] - 1227s 2s/step - loss: 0.4327 - acc: 0.7964
Epoch 3/10
781/781 [==============================] - 1508s 2s/step - loss: 0.3400 - acc: 0.8477
Epoch 4/10
781/781 [==============================] - 966s 1s/step - loss: 0.2369 - acc: 0.8999
Epoch 5/10
368/781 [=============>................] - ETA: 8:29 - loss: 0.1643 - acc: 0.9332